In [17]:
#Isiimporuojame tai ko mums prireiks vėliau 
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker


In [18]:
years = list(range(2001, 2022))
url_data = "https://admin.euroleague.net/main/statistics?mode=Leaders&entity=Players&seasonmode=Single&seasoncode=E{}{&cat=Score&agg=PerGame"

In [19]:
#Susikuriame mums reikalingą katalogą admin_euroleague ir į jį parsisiunčiame html failus per kuriuos iteruosime vėliau , tai daroma norint išvengti pastovaus kreipimosi į puslapį ir galimo duomenų pakitimo
def get_htmls(url_data):
    for year in years:
        url = url_data.format(year)
        data = requests.get(url)
    with open("scoring_leaders_htmls/{}.html".format(year), "w+" , encoding="utf-8") as file :
        file.write(data.text)


In [20]:
#Nusiskaitome parsisiūstus html failus iš admin_euroleague folderio
with open ("scoring_leaders_htmls/2003.html",encoding="utf-8") as f :
    page =f.read()

In [21]:
#Priskiriame konstantą ir nurodome kokį parserį naudosime
soup = BeautifulSoup(page, "html.parser")


In [22]:
#Nurodome mus dominančios lentelės klasę ir jai priskirtą ID
soup.find('tr',class_="StatsHeaderGridResults")
statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")

In [23]:
#Pasitikriname ar gavome mus dominančius duomenis
leaders_2003 = pd.read_html(str(statistics_table))[0]
leaders_2003

,Rank,Player,Team,GamesG,PointsPTS,Ave.,Per 40 min.40m
0,1,"BROWN, MARCUSBROWN, M.",CSKCSKA MoscowCSKA Moscow,21,393,18.71,21.86
1,2,"MACIJAUSKAS, ARVYDASMACIJAUSKAS, A.",BKNTau CeramicaTau Ceramica,20,389,19.45,25.47
2,3,"VUJCIC, NIKOLAVUJCIC, N.",MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv,21,353,16.81,19.26
3,4,"GREER, LYNNGREER, L.",WKSIdea SlaskIdea Slask,14,351,25.07,27.97
4,5,"EVANS, MAURICEEVANS, M.",TREBenetton BasketBenetton Basket,20,348,17.40,21.90
5,6,"LIADELIS, PANAGIOTISLIADELIS, P.",OLYOlympiacosOlympiacos,20,339,16.95,23.19
6,7,"PARKER, ANTHONYPARKER, A.",MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv,21,336,16.00,18.21
7,8,"JASIKEVICIUS, SARUNASJASIKEVICIUS, S.",MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv,21,335,15.95,21.47
8,9,"VUJANIC, MILOSVUJANIC, M.",FORSkipper Fortitudo BolognaSkipper Fortitudo ...,20,326,16.30,22.37
9,10,"GRANGER, ANTONIOGRANGER, A.",EFSEfes PilsenEfes Pilsen,20,314,15.70,18.35


In [24]:
#Susikuriame ryšį su duomenų baze
engine = create_engine('sqlite:///2003_leaders.db')
Session = sessionmaker(bind=engine)
session = Session()

In [25]:
#Susikuriame duomenų bazę
leaders_2003.to_sql(
    name = "2003_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

50

In [26]:
#Persikeliame visus išskreipintus duomenis į duomenų bazę
leaders_2003_select = session.execute('select * from "2003_leaders"')
leaders_2003_select.all()

[(0, 1, 'BROWN, MARCUSBROWN, M.', 'CSKCSKA MoscowCSKA Moscow', 21, 393, 18.71, 21.86),
 (1, 2, 'MACIJAUSKAS, ARVYDASMACIJAUSKAS, A.', 'BKNTau CeramicaTau Ceramica', 20, 389, 19.45, 25.47),
 (2, 3, 'VUJCIC, NIKOLAVUJCIC, N.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv', 21, 353, 16.81, 19.26),
 (3, 4, 'GREER, LYNNGREER, L.', 'WKSIdea SlaskIdea Slask', 14, 351, 25.07, 27.97),
 (4, 5, 'EVANS, MAURICEEVANS, M.', 'TREBenetton BasketBenetton Basket', 20, 348, 17.4, 21.9),
 (5, 6, 'LIADELIS, PANAGIOTISLIADELIS, P.', 'OLYOlympiacosOlympiacos', 20, 339, 16.95, 23.19),
 (6, 7, 'PARKER, ANTHONYPARKER, A.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv', 21, 336, 16.0, 18.21),
 (7, 8, 'JASIKEVICIUS, SARUNASJASIKEVICIUS, S.', 'MACMaccabi Elite Tel AvivMaccabi Elite Tel Aviv', 21, 335, 15.95, 21.47),
 (8, 9, 'VUJANIC, MILOSVUJANIC, M.', 'FORSkipper Fortitudo BolognaSkipper Fortitudo Bologna', 20, 326, 16.3, 22.37),
 (9, 10, 'GRANGER, ANTONIOGRANGER, A.', 'EFSEfes PilsenEfes Pilsen', 20, 314

In [27]:
#Prasiiteruojame per visus gautus html failus ir išsitraukiame duomenis iš visų likusių sezonų ir pridedame prie duomenų 2003 metų statistiką kurią išsitraukėme anksčiau
dfs = []
for year in years :
    with open ("scoring_leaders_htmls/{}.html".format(year),encoding="utf-8") as f :
            page =f.read()
            soup = BeautifulSoup(page, "html.parser")
            soup.find('tr',class_="StatsHeaderGridResults")
            statistics_table=soup.find(id="ctl00_ctl00_ctl00_ctl00_maincontainer_maincontent_contentpane_ctl01_ctl01_gvResults")
            leaders_2003= pd.read_html(str(statistics_table))[0]
            leaders_2003["Year"] = year
            dfs.append(leaders_2003)

In [28]:
#Susjungiame visus duomenis ir pasitikriname ar tikrai juos visus gavome
leaders = pd.concat(dfs)
leaders

,Rank,Player,Team,GamesG,PointsPTS,Ave.,Per 40 min.40m,Year
0,1,"FORD, ALPHONSOFORD, A.",OLYOlympiacosOlympiacos,20,495,24.75,28.09,2001
1,2,"BODIROGA, DEJANBODIROGA, D.",PAOPanathinaikos AthensPanathinaikos Athens,22,441,20.05,24.83,2001
2,3,"GIRICEK, GORDANGIRICEK, G.",CSKCSKA MoscowCSKA Moscow,18,413,22.94,26.76,2001
3,4,"FUCKA, GREGORFUCKA, G.",FORSkipper BolognaSkipper Bologna,20,381,19.05,23.88,2001
4,5,"AVLEEV, RUSLANAVLEEV, R.",PEMUral GreatUral Great,20,372,18.60,24.90,2001
...,...,...,...,...,...,...,...,...
45,46,"DOBRIC, OGNJENDOBRIC, O.",CZVCrvena Zvezda mts BelgradeCrvena Zvezda mts...,29,299,10.31,18.25,2021
46,47,"CLYBURN, WILLCLYBURN, W.",CSKCSKA MoscowCSKA Moscow,20,298,14.90,21.14,2021
47,47,"PAPAGIANNIS, GEORGIOSPAPAGIANNIS, G.",PAOPanathinaikos OPAP AthensPanathinaikos OPAP...,29,298,10.28,14.47,2021
48,49,"KURIC, KYLEKURIC, K.",BARFC BarcelonaFC Barcelona,37,292,7.89,15.68,2021


In [13]:
#Šią kodo dalį ja matėmė šiektiek anksčiau
leaders.to_sql(
    name = "all_scoring_leaders",
    con = engine,
    index = True,
    if_exists = "replace"
)

1050

In [14]:
leaders_select = session.execute('select * from"all_scoring_leaders"')
leaders_select.all()

[(0, 1, 'FORD, ALPHONSOFORD, A.', 'OLYOlympiacosOlympiacos', 20, 495, 24.75, 28.09, 2001),
 (1, 2, 'BODIROGA, DEJANBODIROGA, D.', 'PAOPanathinaikos AthensPanathinaikos Athens', 22, 441, 20.05, 24.83, 2001),
 (2, 3, 'GIRICEK, GORDANGIRICEK, G.', 'CSKCSKA MoscowCSKA Moscow', 18, 413, 22.94, 26.76, 2001),
 (3, 4, 'FUCKA, GREGORFUCKA, G.', 'FORSkipper BolognaSkipper Bologna', 20, 381, 19.05, 23.88, 2001),
 (4, 5, 'AVLEEV, RUSLANAVLEEV, R.', 'PEMUral GreatUral Great', 20, 372, 18.6, 24.9, 2001),
 (5, 6, 'GINOBILI, EMANUELGINOBILI, E.', 'VIRKinder Virtus BolognaKinder Virtus Bologna', 22, 345, 15.68, 22.03, 2001),
 (6, 7, 'HOLDEN, JRHOLDEN, JR', 'AEKAEK AthensAEK Athens', 20, 344, 17.2, 19.26, 2001),
 (7, 8, 'BOOKER, MELVINBOOKER, M.', 'PESScavolini PesaroScavolini Pesaro', 20, 343, 17.15, 20.31, 2001),
 (8, 9, 'EDNEY, TYUSEDNEY, T.', 'TREBenetton BasketBenetton Basket', 19, 341, 17.95, 23.66, 2001),
 (9, 10, 'KAMBALA, KASPARSKAMBALA, K.', 'EFSEfes PilsenEfes Pilsen', 20, 338, 16.9, 22.32, 2